# data preprcessing step1
이 파일은 대용량 데이터를 저장해야 하기 때문에 colab 환경에 아닌, 로컬 환경에서 진행.


In [ ]:
import os
import numpy as np
import pandas as pd
import json
import sys
import random
import copy
import transformers
import shutil
import tqdm

from tqdm import tqdm

In [ ]:
def get_dir_size(path): #전체 데이터의 크기를 확인
    total = 0
    with os.scandir(path) as it:
        for entry in it:
            if entry.is_file():
                total += entry.stat().st_size
            elif entry.is_dir():
                total += get_dir_size(entry.path)
    return total

In [ ]:
def Create_token_json_file(next_num_file=10000, random_num_file=10000):
  next_pred_json_list = []
  random_pred_json_list = []
  if os.path.isdir('/Users/yoon-seunggyu/conda_folder/next_pred') == False:
    os.mkdir('/Users/yoon-seunggyu/conda_folder/next_pred') #다음 문장 예측에서 올바르게 예측된 문장들의 집합 폴더
  if os.path.isdir('/Users/yoon-seunggyu/conda_folder/random_pred') == False:
    os.mkdir('/Users/yoon-seunggyu/conda_folder/random_pred') #다음 문장 예측에서 랜덤하게 연결된 문장들의 집합 폴더
  for i in range(next_num_file):
    next_pred_json_list.append(f'/Users/yoon-seunggyu/conda_folder/next_pred/{i}.json') #입력된 갯수만큼의 json 파일을 생성
  for i in range(random_num_file):
    random_pred_json_list.append(f'/Users/yoon-seunggyu/conda_folder/random_pred/{i}.json')

  return next_pred_json_list, random_pred_json_list

In [ ]:
def Init_json_file(file):
  news_label = {}
  news_label['sentence'] = []
  with open(file, 'w') as f:
    json.dump(news_label, f, indent='\t')

In [ ]:
def Crete_mask(sentence):
  mask_sen = copy.deepcopy(sentence)
  label_lm = []
  for index, en in enumerate(mask_sen):
    if index == 0 or index == len(mask_sen) - 1: #[cls], [sep]는 넘어가야함
      label_lm.append(-1)
      continue
    if random.random() < 0.2: #masking prob: 20%
      if random.random() < 0.7:
        mask_sen[index] = 4 #mask token index: 4
      else:
        if random.random() > 0.5: #다른 단어로 교체
          mask_sen[index] = int(random.random() * 32200)
      label_lm.append(sentence[index])
    else:
      label_lm.append(-1)
  return mask_sen, label_lm

In [ ]:

"""
2개 문장 단위로 불러옴.
불러온 인덱스가 2로 나눌시 나머지가 0일 경우는 next json 폴더에 저장됨.
이 경우 두 문장 모두 같은 폴더에 순서대로 저장.
나머지가 1일 경우는 random json 폴더에 저장.
이 경우는 다른 2 파일을 불러와 따로 저장함(겹치지 않게 하기 위해).
파일이 중복될 확률이 1/random_num_file**2이기 때문에, 확인 코드 없이 진행할 예정
news의 길이가 홀수 인 경우, 마지막 문장은 random json 폴더에 저장.
"""
def Add_sentence_to_new_json(news, news_label_lms, next_pred_json_list, random_pred_json_list, next_num_file=10000, random_num_file=10000):
  news_len = len(news) // 2
  odd_bool = True if len(news)%2 == 1 else False
  if odd_bool == True: ## news의 길이가 홀수일 경우, random predict로 보내기
    file_index = int(random.random() * random_num_file)
    news_odd = news[-1]
    label_odd = news_label_lms[-1]
    data = (news_odd, label_odd)
    with open(random_pred_json_list[file_index], 'r') as r_save_file: #read save_file
      save_json = json.load(r_save_file)
    save_json['sentence'].append(data)
    with open(random_pred_json_list[file_index], 'w') as save_file: #save_file
      json.dump(save_json, save_file)

  for i in range(news_len):
    if i % 2 == 0: #next-sentence json으로
      file_index = int(random.random() * next_num_file)
      news_next1 = news[i*2]
      label_next1 = news_label_lms[i*2]
      news_next2 = news[i*2 +1]
      label_next2 = news_label_lms[i*2 +1]
      data1 = [news_next1, label_next1]
      data2 = [news_next2, label_next2]
      data = (data1, data2)

      with open(next_pred_json_list[file_index], 'r') as r_save_file:
        save_json = json.load(r_save_file)
      save_json['sentence'].append(data)
      with open(next_pred_json_list[file_index], 'w') as save_file:
        json.dump(save_json, save_file)

    else:
      file_index1 = int(random.random() * random_num_file)
      file_index2 = int(random.random() * random_num_file)
      news_next1 = news[i*2]
      label_next1 = news_label_lms[i*2]
      news_next2 = news[i*2 +1]
      label_next2 = news_label_lms[i*2 +1]
      data1 = (news_next1, label_next1)
      data2 = (news_next2, label_next2)

      #2개의 데이터를 연결되지 않게 하기 위해 다른 폴더에 저장. 체크를 따로 하지 않는 이유는 중복되서 저장될 확률이 1/random_num_file**2이기 때문. 
      with open(random_pred_json_list[file_index1], 'r') as r_save_file1:
        save_json1 = json.load(r_save_file1)
      with open(random_pred_json_list[file_index2], 'r') as r_save_file2:
        save_json2 = json.load(r_save_file2)
      save_json1['sentence'].append(data1)
      save_json2['sentence'].append(data2)
      with open(random_pred_json_list[file_index1], 'w') as save_file1:
        json.dump(save_json1, save_file1)
      with open(random_pred_json_list[file_index2], 'w') as save_file2:
        json.dump(save_json2, save_file2)

In [ ]:
size_result = get_dir_size("/Users/yoon-seunggyu/Downloads/030.웹데이터 기반 한국어 말뭉치 데이터/01.데이터/1.Training/라벨링데이터")
print(str(size_result / 1000 ** 3) + "Gb") #gb

28.961090787Gb


In [ ]:
vocab = transformers.BertTokenizer('/Users/yoon-seunggyu/Downloads/vocab.txt', do_lower_case=False, strip_accents=False)

In [ ]:
label_json_path = '/Users/yoon-seunggyu/Downloads/030.웹데이터 기반 한국어 말뭉치 데이터/01.데이터/1.Training/라벨링데이터/TL1'
label_list_dir = os.listdir(label_json_path)
label_list_dir.remove('.DS_Store')

In [ ]:
label_list_dir

label_dict = {}
for label_dir in label_list_dir:
  label_dict[label_dir] = os.listdir(label_json_path + '/' + label_dir)

In [ ]:
vocab.vocab_size

32200

In [ ]:
next_num_file = 10000
random_num_file = 10000
next_pred_json_list, random_pred_json_list = Create_token_json_file(next_num_file, random_num_file)

In [ ]:
for n_pred, r_pred in zip(next_pred_json_list, random_pred_json_list):
  Init_json_file(n_pred)
  Init_json_file(r_pred)

In [ ]:
sum_file = 0
for key, values in label_dict.items(): #종류 단위
  print(key + ": " + str(len(values)) + '(json파일 갯수)')
  sum_file += len(values)
print("총 파일 갯수: " + str(sum_file))

취미: 2746(json파일 갯수)
스포츠: 2550(json파일 갯수)
산업: 4031(json파일 갯수)
지역: 3296(json파일 갯수)
정치: 3286(json파일 갯수)
건강: 3247(json파일 갯수)
IT_과학: 1217(json파일 갯수)
사회일발: 3474(json파일 갯수)
경제: 5072(json파일 갯수)
국제: 2162(json파일 갯수)
여성복지: 2786(json파일 갯수)
교육: 2789(json파일 갯수)
문화: 1683(json파일 갯수)
여행레저: 3162(json파일 갯수)
연예: 4603(json파일 갯수)
라이프스타일: 2913(json파일 갯수)
사건사고: 2813(json파일 갯수)
총 파일 갯수: 51830


In [ ]:
for key, values in label_dict.items(): #종류 단위
  print(key + ": " + str(len(values)) + '(json파일 갯수)')
  with tqdm(total=len(values), desc=f"{key} 진행도: ") as pbar:
    for value_len, value in enumerate(values): #기사가 포함되어 있는 파일 하나씩
      with open(label_json_path + '/' + key + '/' + value, 'r') as f: 
        json_data = json.load(f)
        len_named = len(json_data['named_entity']) #-> 0 ~ len_named // 2 -1 안의 무작위 값을 random으로 받아오기(vocab으로 변환 후 길이로 해야함)
        for named_entity in json_data['named_entity']: #한 파일안의 기사 하나씩 꺼내기
          contents = named_entity['content']
          news = []
          temp_news = []
          news_label_lms = []
          temp_news_label_lms = []
          for content in contents: #기사 내의 한문장씩
            sentence = content['sentence'] # -> vocab으로 변환하여, 255까지 채울 수 있도록 하기(bos, eos, sep 토큰도 들어가야하기 때문)
            sentence = vocab.encode(sentence)
            sentence, label_lm = Crete_mask(sentence)
            if len(sentence) + len(temp_news) > 254:
              temp_news += [3] #[sep] token 붙이기 
              temp_news_label_lms += [-1]
              news.append(temp_news)
              news_label_lms.append(temp_news_label_lms)
              temp_news = []
              temp_news_label_lms = []
              temp_news += sentence[:-1]
              temp_news_label_lms += label_lm[1:-1]
            else:
              temp_news += sentence[1:-1]
              temp_news_label_lms += label_lm[1:-1]
          Add_sentence_to_new_json(news, news_label_lms, next_num_file, random_num_file, next_pred_json_list, random_pred_json_list)
      pbar.update(1)